In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional
requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem
with real data. The data is given in the file diet.xls.

### Question 15.2.1
Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the
maximum and minimum daily nutrition constraints, and solve it using PuLP. Turn in your code
and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs,
oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)

In [7]:
from pulp import *
import pandas as pd

# Read data
data=pd.read_excel("dietSummer2018.xls")
#print(data)
data=data[0:64]

#Convert dataframe to list
data=data.values.tolist()
#for i in range(len(data)):
#    print(data[i])
#print(data)

#Extract vectors of data for each nutrient
foods = [x[0] for x in data] #list of food names
calories = dict([(x[0], float(x[3])) for x in data]) #Total calories for each food
totalFat = dict([(x[0], float(x[5])) for x in data]) #Total fat for each food



#Cost of each food
cost = dict([(x[0], float(x[1])) for x in data]) 


#Create minimization problem
problem=LpProblem("15.2.1",LpMinimize)

#Define variables
chosenVars=LpVariable.dicts("Chosen",foods,0,1,'Binary')
x=LpVariable.dicts("x",foods,0)

#Create objective function
problem += lpSum([cost[f] * x[f] for f in foods])

#Add constraints for each nutrient

#Create lower/upper bounds 
#Create constraint matrix
amin=[1500,30,20,800,130,125,60,1000,400,700,10]
amax=[2500,240,70,2000,450,250,100,10000,5000,1500,40]
B=[]
for i in range(0,11):
    B.append(dict([(x[0], float(x[i+3])) for x in data]))
for i in range(0,11):
    dot_B_x = pulp.lpSum([B[i][j] * x[j] for j in foods])
    condition = amin[i] <= + dot_B_x
    problem += condition
    
    dot_B_x = pulp.lpSum([B[i][j] * x[j] for j in foods])
    condition = amax[i] >= + dot_B_x
    problem += condition



#Solve the optimization problem
problem.solve()
#print(problem.variables())

for var in problem.variables():
    if var.varValue>0:
        if str(var).find('Chosen'):
            print(str(var.varValue)+" units of " + str(var) )
print("Total cost of food = $%.2f" % value(problem.objective))








52.64371 units of x_Celery,_Raw
0.25960653 units of x_Frozen_Broccoli
63.988506 units of x_Lettuce,Iceberg,Raw
2.2929389 units of x_Oranges
0.14184397 units of x_Poached_Eggs
13.869322 units of x_Popcorn,Air_Popped
Total cost of food = $4.34


### Question 15.2.2

Please add to your model the following constraints (which might require adding more variables)
and solve the new model:

a. If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will
need two variables for each food i: whether it is chosen, and how much is part of the diet.
You’ll also need to write a constraint to link them.)

b. Many people dislike celery and frozen broccoli. So at most one, but not both, can be
selected.

c. To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be
selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered 
meat?), just call it whatever you think is appropriate – I want you to learn how to write this
type of constraint, but I don’t really care whether we agree on how to classify foods!]


In [10]:
from pulp import *
import pandas as pd
import numpy as np
import random

# Read data
data=pd.read_excel("dietSummer2018.xls")
data=data[0:64]

#Convert dataframe to list
data=data.values.tolist()

#Extract vectors of data for each nutrient
foods = [x[0] for x in data] #list of food names
calories = dict([(x[0], float(x[3])) for x in data]) #Total calories for each food
totalFat = dict([(x[0], float(x[5])) for x in data]) #Total fat for each food


#Cost of each food
cost = dict([(x[0], float(x[1])) for x in data]) 

#Create minimization problem
problem2=LpProblem("15.2.2",LpMinimize)

#Define variables
chosenVars=LpVariable.dicts("Chosen",foods,0,1,cat='Binary')
x=LpVariable.dicts("x",foods,0)
foodVars = LpVariable.dicts("Foods", foods, 0, cat='Continuous') 
#foodVars represents how much of each food we eat/consume
chosen=dict()
#print(np.where(foodVars[f]>=0.0001, 1, 0))

#Create objective function
problem2 += lpSum([cost[f] * x[f] for f in foods])


#Add constraints for each nutrient

#Create lower/upper bounds 
amin=[1500,30,20,800,130,125,60,1000,400,700,10]
amax=[2500,240,70,2000,450,250,100,10000,5000,1500,40]

#Add constraints on the amount of food that you must select
for f in foods:
    problem2 += foodVars[f] <= 1000000*chosenVars[f]
    problem2 += foodVars[f] >= 0.1*chosenVars[f]

#15.2.2 part b
problem2 +=chosenVars['Celery, Raw']+chosenVars['Frozen Broccoli']<=1

#Create constraint matrix
B=[]
for i in range(0,11):
    B.append(dict([(x[0], float(x[i+3])) for x in data]))
for i in range(0,11):
    dot_B_x = pulp.lpSum([B[i][j] * x[j] for j in foods])
    condition = amin[i] <= + dot_B_x
    problem2 += condition
    condition = amax[i] >= + dot_B_x
    problem2 += condition

#Add constraints tying foodVars to chosenVars
for f in foods:
    problem2 += chosenVars[f]<=foodVars[f]
    if foodVars[f]>=0.0001:
        problem2 += chosenVars[f]==1
    if foodVars[f]==0:
        problem2 += chosenVars[f]==0
    if chosenVars[f]==0:
        problem2 += foodVars[f]==0



#15.2.2 part c
protein_foods=['Roasted Chicken','Poached Eggs','Scrambled Eggs','Bologna,Turkey','Frankfurter, Beef','Ham,Sliced,Extralean',
               'Kielbasa,Prk','Taco','Hamburger W/Toppings','Hotdog, Plain','Peanut Butter','Pork','Sardines in Oil',
               'White Tuna in Water','Chicknoodl Soup','New E Clamchwd,W/Mlk']
rand=random.sample(protein_foods, 3)

print(rand)
problem2+=chosenVars[rand[0]]+chosenVars[rand[1]]+chosenVars[rand[2]]>=3

#Solve the optimization problem
problem2.solve()
      
for var in problem2.variables():
    if var.varValue>0:
        if str(var).find('Chosen'):
            print(str(var.varValue)+" units of " + str(var) )
print("Total cost of food = $%.2f" % value(problem2.objective))



['Pork', 'Hamburger W/Toppings', 'Chicknoodl Soup']
1.0 units of Foods_2%_Lowfat_Milk
1.0 units of Foods_3.3%_Fat,Whole_Milk
1.0 units of Foods_Apple,Raw,W_Skin
1.0 units of Foods_Apple_Pie
1.0 units of Foods_Bagels
1.0 units of Foods_Banana
1.0 units of Foods_Beanbacn_Soup,W_Watr
1.0 units of Foods_Bologna,Turkey
1.0 units of Foods_Butter,Regular
1.0 units of Foods_Cap'N_Crunch
1.0 units of Foods_Carrots,Raw
1.0 units of Foods_Celery,_Raw
1.0 units of Foods_Cheddar_Cheese
1.0 units of Foods_Cheerios
1.0 units of Foods_Chocolate_Chip_Cookies
1.0 units of Foods_Corn_Flks,_Kellogg'S
1.0 units of Foods_Couscous
1.0 units of Foods_Crm_Mshrm_Soup,W_Mlk
1.0 units of Foods_Frankfurter,_Beef
1.0 units of Foods_Frozen_Broccoli
1.0 units of Foods_Frozen_Corn
1.0 units of Foods_Grapes
1.0 units of Foods_Ham,Sliced,Extralean
3.0 units of Foods_Hamburger_W_Toppings
1.0 units of Foods_Hotdog,_Plain
1.0 units of Foods_Kielbasa,Prk
1.0 units of Foods_Kiwifruit,Raw,Fresh
1.0 units of Foods_Lettuce,Iceb